# Mini Curso de Otimização em Julia

## Dica 1: Prepare seu Ambiente de Desenvolvimento 

 Donald Knuth disse `Premature Optimization is the Root of All Evil`, essa afirmação é especialmente verdade quando você não sabe onde otimizar, e para isso o `VSCode` será seu aliado. 

 As extensões recomendadas são: 
 - `Julia Language Support`: Essencial
 - `Flame Chart Visualizer`: Profiler é gerado com essa extensão
 - `Fast Unicode Math Characters`: Para adicionar simbolos de vetores corretamente
 - `XT Color Theme`: Sugestão pessoal

É recomendado que você *crie* ou *ative* o seu local de desenvolvimento. Para isso você precisa de um arquivo `Manifest.toml` dizendo a versão de todos os pacotes que estará utilizando. Isto é equivalente aos `Virtual Enviroments` do Conda.

A [documentação](https://pkgdocs.julialang.org/v1/creating-packages/index.html) nos diz o caminho. Apenas os comandos são o suficiente.

In [1]:
import Pkg; 
Pkg.generate("ERAD2022")
cd("ERAD2022")
Pkg.activate(".")

Hello World!

  Generating  project meuModulo:
    meuModulo/Project.toml
    meuModulo/src/meuModulo.jl
  Activating project at `~/Documents/ERAD2022/meuModulo`


In [6]:
using ERAD2022
ERAD2022.greet()

Esse esforço adicional para programar é útil quando você for criar programas mais longos, pois o seu `Módulo` poderá ser *escaneado* pelo pacote `Revise`, que irá recompilar as funções que você atualizar automaticamente.

## Dica 2: Instale MKL.jl corretamente

Computação científica faz forte uso de Algebra Linear, e por padrão, Julia vem com `OpenBlas` instalado. No entanto, sua concorrente direta é a biblioteca *intel MKL*, que é conhecida por ser mais rápida.  Após muito tempo no [Julia Discourse](https://discourse.julialang.org/) acredito que essa afirmação não é 100% correta, existem bibliotecas em Julia tão rapidas quando o MKL, ou ainda regimes no qual o OpenBlas é mais rápido. *Você terá que testar e descobrir se sua aplicação é mais rápida com MKL*.

A partir da *versão v1.7* a instalação do MKL é muito prática:

In [ ]:
import Pkg; Pkg.add("MKL")
Pkg.status("MKL");

**CONTUDO:** **No Linux**, ainda é necessário configurar as variáveis do ambiente para fazer uso adequado do multithreading. Como MKL faz uso de `OpenMP`, você tem que informar quantas threads serão utilizadas definindo a variável de sistema `OMP_NUM_THREADS`. O melhor desempenho acontece quando numero de threads é igual ao número de processadores **fisicos**. Contudo, essa flag não permite Julia utilizar todas as threads disponiveis, por isso você precisa exportar a flag `JULIA_NUM_THREADS` seperadamente.

Meu computador possui 4 Cores e 8 Threads, então eu declaro as variaveis no arquivo `~/.bashrc`:
```
export JULIA_NUM_THREADS=8
export OMP_NUM_THREADS=4
```

Nos meus testes, não tive variação de desempenho no Windows.

Agora, você pode inicie Julia novamente, e execute `using MKL`. A seguir, testaremos o desempenho do OpenBlas contra MKL para diferentes tamanhos de matrizes. Caso não tenha instalado, adicione o pacote `Plots`.

In [ ]:
import Pkg; Pkg.add(["Plots", "Random"]);

In [ ]:
using LinearAlgebra, Random, Plots

"""
    resolver sistemas de equações lineares (Ax=b)
"""
function testandoMKL(N)
    A = rand(N, N)
    b = rand(N)
    x = @elapsed  A\b
    return x
end
testandoMKL(5) # warm up
testandoMKL(10) # warm up

In [ ]:
N_em_escala_log = round.(Int, 10.0.^range(2, 3, length=100))

Random.seed!(2022)
t_openblas = Float64[]
@show BLAS.get_config() # verificar que é o OpenBLAS
for N = N_em_escala_log
    push!(t_openblas, testandoMKL(N))
end

In [ ]:
using MKL # agora o MKL vai funcionar
Random.seed!(2022)
t_mkl = Float64[]
@show BLAS.get_config()
for N = N_em_escala_log
    push!(t_mkl, testandoMKL(N))
end;

In [ ]:
plot(N_em_escala_log, t_openblas,label="OpenBlas", markershape=:square, legend=:topleft)
plot!(N_em_escala_log, t_mkl,label="MKL", scale=:log10, markershape=:circle)
xlabel!("N")
ylabel!("tempo [s]")

## Dica 3: Declare os Tipos
 
 Julia tem otimizações automáticas e elas são mais efetivas quando os *tipos* dos objetos são bem definidos. Dê preferência a `Arrays` e `Structs`, e evite `Dict`. A seguir alguns exemplos de fixação:

  

In [ ]:
# `a` é Array que pode conter qualquer objeto, por isso, seu tipo é 'Any'.
a = []
push!(a, 2)
push!(a, 3.0)
println("Tipo de a: ", typeof(a))

# Se temos certeza que iremos armazenar apenas valores do tipo `Float64`, basta declará-lo antes do `[]`.
b = Float64[]
push!(b, 2) # Julia fará a conversão de `2` para `2.0`
push!(b, 3)
println("Tipo de b: ",typeof(b))

In [ ]:
push!(a, 4 + 5im) # funciona
@show a;
println("-------")
try
    push!(b, 4 + 5im) # não funciona
catch
    println("Não existe conversão de Float para Complex.")
    @show b
end;

*structs* são fáceis de criar, e aqui trago um alerta. Use *immutable structs* sempre que possível. Isso permitirá ao otimizador de Julia fazer seu serviço de forma mais eficiente, já que os tipos não mudam. 

A otimização com `struct` mais interessante é alocar dados na memória *stack* e não na *heap*. Para você não se preocupar em como fazer isso, já existe o pacote `StaticArrays`.   
Importante: esse pacote só é útil para vetores ou matrizes pequenos - 'pequeno' depende do contexto, o mais comum é armazenar coordenadas (x,y,z).

In [ ]:
Pkg.add(["StaticArrays", "BenchmarkTools"])
using StaticArrays, BenchmarkTools

v_default = [[i^2, i^3, 4i] for i = 1:1000]
v_static =  [@SVector [i^2, i^3, 4i] for i = 1:1000]

@btime sum.(v_default)
@btime sum.(v_static);

Outro problema comum é que Julia pode ficar indeciso sobre qual tipo será utilizado, é o famoso **type instability problem**.  Recomendo o post [Writing type-stable Julia code](https://www.juliabloggers.com/writing-type-stable-julia-code) ou [Type instability and performance](https://m3g.github.io/JuliaNotes.jl/stable/instability/). 

Na prática, minha dica é usar `typeof` sempre que possível, o caso mais recorrente é a inicialização de contadores. No lugar de escrever `u = 0` ou `u = 0.0`, permita que Julia crie a variável do tipo adequada, usando a função  `zero`.

In [ ]:
v = rand(ComplexF32, 10)

# Não faço isso, mesmo estando correto
u_sum = 0f0 + 0f0*im
u_vec = ComplexF32[]

# Faça isso. Assim seu código é genérico
u_sum = zero( eltype(v) ) # == zero(ComplexF32) == 0.f0 + 0.f0im
u_vec = eltype(v)[] # Array do tipo ComplexF32

for vv in v
    u_sum += vv
    push!( u_vec, 2*vv)
end
@show u_sum
@show u_vec;


## Dica 4: Faça o Benchmark corretamente

O que não falta na internet são posts de pessoas dizendo que seus código em Julia estão lentos, e logo em seguida, alguém aponta que o processo de medição esta incorreto. 

O tempo de um código em Julia é composto do **Tempo de Compilação** e o **Tempo de Execução**. O tempo predominante depende muito do código e da escala do problema. O que você realmente quer medir, é o Tempo de Execução, e isso não é possivel de fazer usando a função `time` no Bash do Linux (se você fizer isso, irá medir ambos os tempos). A solução é:
- Crie seu código
- Faça um *aquecimento*: execute ele 1 vez, porém com parâmetros pequenos, apenas para gerar o código de máquina
- Execute novamente seu código, com parâmetro realisticos, e meça o tempo de execução
    - Se o tempo for na ordem de segundos, use a macro `@elapsed`
    - Se o tempo for curto, utilize um pacote especializado chamado `BenchmarkTools`. Com ele, use a macro `@belapsed`, e não esqueça de usar `$` na chamada de função
- Salve o tempo de execução em um arquivo, ou exiba na tela


In [ ]:
x = rand(8_000_000)
tempo1 = @elapsed sum(x);


using BenchmarkTools
tempo2 = @belapsed sum($x);

@show tempo1, tempo2

Caso esteja programando de forma interativa, vale a pena conferir quanta memória esta sendo alocada usando `@time` - normalmente, quanto menos memória, melhor. Entretando, é mais relevante você ter uma ideia completa do tempo de execução, e para isso `BenchmarkTools` exporta a função `@benchmark`. Nela você vai reparar que o tempo emitido pelo `@belapsed` foi o tempo minimo entre várias repetições.

In [ ]:
@benchmark sum($x)

## Dica 5: Crie Funções

Na *Dica 2* eu fiz questão de criar a função `testandoMKL` para medir o tempo de execução, e isso não foi uma conveniência ou estética para os meus códigos. O *Escopo Global* (seu REPL) de Julia precisa garantir que seus códigos funcionem ao custo de não garantir os tipos - tipos genéricos `Any` não têm muitas otimizações. 

**A solução é simples: Crie Funções**

In [ ]:
@time begin
    N = 1_000
    A = zeros(N,N)
    for m=1:N
        for n=1:N
            A[n,m] = rand(eltype(A))
        end
    end
end

function criei_função()
    N = 1_000
    A = zeros(N,N)
    for m=1:N
        for n=1:N
            A[n,m] = rand(eltype(A))
        end
    end
end
@time criei_função()

Claro que criar funções em todos os lugares nem sempre é a melhor solução, e você gostaria que o compilador fizesse `inline expansion`. Você pode indicar isso manualmente para o compilador colocando a macro `@inline` antes da declaração da função.

In [ ]:
@inline function calculoPrincipal()
    # fazer algo
end

function funçãoPrincipal()
    for i=1:N
        calculoPrincipal()
    end
end

# Projetos

# Projeto 1: Transcrição Python em Julia

Agradecimentos ao Edmilson Roque (Cris), ICMC-USP por fornecer o código.

Faremos uma conversão de código que realiza Processo de Ortogonalização de Gram-Schmidt, cujos vetores são `Funções`, e não `Arrays`. 

O código original faz uso de recursividade e funções anônimas (função lambda). Essas grande quantidade de chamadas de funções será contornada com Memoização e Cálculo Simbólico.



# Projeto 2: Evolução de EDOs

Faremos a evolução temporal do conjunto de equações diferencais que representam a excitação de estado atômicos
$$ \frac{d\beta_j}{dt} = \left( iΔ - \frac{\Gamma}{2}\right )\beta_j + \frac{i\Omega_j}{2}z_j - z_j\sum_{m ≠ j}^N G_{j,m}\beta_m$$
$$ \frac{dz_j}{dt} = [i\Omega_j ^\dagger\beta_j - i\Omega_j\beta_j ^\dagger] - \Gamma (1+z_j) + \frac{2}{\Gamma}\sum_{j≠ m}^N G_{j,m}\beta_m \beta_j^\dagger$$

Com $\Gamma=1$, $G = -\frac{\Gamma}{2}exp(-ir_{jm})/ir_{jm}$ e condição inicial $\beta_0 = 0$ e $z_0 = -1$


Algumas otimizações não são limitadas a Julia, mas são educativas para o público geral. Nesse projeto desejamos evitar alocação de memória (variáveis temporárias) com @views e multiplicação matricial. Além disso, poderemos dar nosso primeiro código em GPU.

# Projeto 3: Somatória Dupla

Uma vez que calcumos os estados atômicos, $\beta_n$, do projeto anterior, uma pergunta natural é descobrir qual a intesidade da luz emitida em certa direção do espaço $\hat{n}$. Para esse projeto, a intensidade será calculada pela fórmula 

$$ I = \sum_n^N \sum_{m ≠ n}^N \beta_n^† \beta_m e^{i\; \hat{n}⋅(\vec{r}_n - \vec{r}_m) } $$

$\vec{r}_n$ é a posição do emissor em coordenadas cartesianas.

Essa somatória dupla possui muito espaço para otimização ao evitar operações desnecessárias, além de ser o caso ideal para multithreading.

# Projeto 4: Processamento de Imagem

Implementaremos um algoritmo muito simples para detecção de bordas em imagens. 
- Analisaremos uma figura pixel-a-pixel, e comparamos a intensidade de cor (média dos canais R,G,B) do pixel atual, com um pixel a esquerda e um pixel inferior.   
    - Se a diferença de valores entre os 3 pontos for maior que um threshold definido pelo usuário (que dependa da imagem)
        - nós pintamos o pixel da borda com uma cor escura
    - Senão for uma borda, deixamos como cor branca


Esse algoritmo é Vergonhosamento Paralelo, e por isso vamos usar o paralelismo nativo de Julia para aprender algumas ferramentas básicas.